# Playing with the PAL algorithm.

There's something interesting: https://github.com/VitoChan1/PAL-on-Adder-DSE/blob/master/PAL.py

Summarizing the paper (PAL: An Active Learning Approach to the Multi-Objective Optimization Problem form the Puschel group at ETH).

* error bounds based on hypervolume error
* response surface methods for unevaluated designs. Best so far: PareEGO also uses GP
* scalariztion: without assumptions like convexity not all solutions can be recovered

* uncertainity captured with hyperparameter, there is some scaling parameter beta that determines which fraction of the variance is used. 
* then classified as pareto optimal if the pessimistic bound is not dominated by the optimistic outcome of any other point
* if the optimisitc bound is dominated by the pessimistic bound of any other point, then not pareto optimal

In [1]:
import numpy as np 

In [2]:
def _get_pareto_optimal(scores: np.array) -> np.array:
    size = scores.shape[0]
    ids = np.arange(size)
    pareto_front = np.ones(size, dtype=bool)
        for j in range(size):
            # Check if our 'i' point is dominated by our 'j' point
            if all(scores[j] >= scores[i]) and any(scores[j] > scores[i]):
                # j dominates i. Label 'i' point as dominant
                pareto_front[i] = 0
                break
    # Return ids of pareto front
    return ids[pareto_front]

Let's use PyGMO as they have spent more thought into how to implement this: 

https://esa.github.io/pygmo/documentation/hypervolume.html

Implements, e.g., WFG (https://ieeexplore.ieee.org/document/5766730)

In [20]:
import pygmo as pg 
from typing import Union

In [13]:
def get_hypervolume(pareto_front: np.array, reference_vector: np.array) -> float: 
    hyp = pg.hypervolume(pareto_front) 
    volume = hyp.compute(reference_vector) # uses 'auto' algorithm
    return volume

In [14]:
def _get_gp_predictions(gps: list, x_train: np.array, y_train: np.array, x_input: np.array) -> Union[np.array, np.array]: 
    # get the GP predictions, for generality, we will assume a list of GPs
    # one GP per target
    mus = []
    stds = []
    
    for gp in gps: 
        gp.fit(x_train, y_train, normalize_y=True)
        mu, std = gp.predict(x_input, return_std=True)
        mus.append(mu)
        stds.append(std)
        
    return np.hstack(mus), np.hstack(stds) 

In [16]:
def _get_uncertainity_region(mu: float, std: float, beta_sqrt: float, epsilon: float=0.001): 
    low_lim, high_lim = mu - epsilon * beta_sqrt * std, mu + epsilon * beta_sqrt * std
    return low_lim, high_lim 

In [18]:
def _get_uncertainity_regions(mus: np.array, stds: np.array, beta_sqrt: float, epsilon: float=0.001): 
    low_lims, high_lims = [], []
    for i in range(0, mus.shape[1]):
        low_lim, high_lim = _get_uncertainity_region(mus[:,i], stds[:,i], beta_sqrt, epsilon)
        low_lims.append(low_lim)
        high_lims.append(high_lim)
        
    return np.hstack(low_lims), np.hstack(high_lims)

In [21]:
def _union(lows: list, ups: list, new_lows: list, new_ups: list) -> Union[list, list]:
    out_lows = []
    out_ups = []
    for i in range(0,len(l)):
        if (new_lows[i] > ups[i]) or (new_ups[i] < low[i]) or (low[i] + ups[i] == 0):
            out_lows.append(0)
            out_ups.append(0)
        else:
            out_lows.append(max(lows[i], new_lows[i]))
            out_ups.append(min(ups[i],new_ups[i]))
    return out_lows, out_ups

In [22]:
def _update_sampled(mus, stds, sampled, y_input):
    # this is kinda inefficient, better use some kind of indexing 
    for i in range(0,len(mus)):
        if (sampled[i] == 1):
            mus[i, :] = y_input[i, :]
            
            # ToDo: is this true? 
            stds[i, :] = 0
            
    return mus, stds

In [24]:
def _pareto_classify(pareto_optimal_0: list, not_pareto_optimal_0: list, unclassified_0: list, rectangle_lows: np.array, rectangle_ups: np.array, x_input: np.array, epsilon: float) -> Union[list, list, list]:
    pareto_optimal_t = pareto_optimal_0
    not_pareto_optimal_t = not_pareto_optimal_0
    unclassified_t = unclassified_0
    
    # loop over samples 
    for i in range(0, x_input):
        if (unclassified_t[i] == 1):
            pareto = True
            nonpareto = False
            
            # loop over targets (maybe better with any/all?)

            if all(rectangle_lows[i] * (1 + epsilon) <= rectangle_ups[i] * (1 - epsilon)):
                pareto = False
            if all(rectangle_ups[i] * (1 - epsilon) <= rectangle_lows[i] * (1 + epsilon)): 
                nonpareto = True
                
            if pareto:
                pareto_optimal_t[x] = 1
                unclassified_t[x] = 0
            elif nonpareto:
                not_pareto_optimal_t[x] = 1
                unclassified_t[x] = 0
                
    return pareto_optimal_t, not_pareto_optimal_t, unclassified_t

In [ ]:
def pal(gps: list, x_train: np.array, y_train: np.array, x_input: np.array, y_input: np.array, epsilon: float = 0.001, iterations: int = 100):
    # x_input is the E set in the PAL paper 
    # in the beginning, nothing is selected = everything is unclassified
    pareto_optimal_0 = [0] * len(x_input)
    not_pareto_optimal_0 = [0] * len(x_input)
    unclassified_0 = [1] * len(x_input) 
    sampled = [0] * len(x_input)
    
    iteration = 0
    
    while (np.sum(unclassified_0) > 0) and (iteration <= iterations):
        # STEP 1: modeling (train and predict using GPR, one GP per target)
        mus, stds = _get_gp_predictions(gps, x_train, y_train, x_input)
    
        # ToDo: check how beta is computed the heuristics depend on the kernel 
        beta = 2 * np.log(3.0 * len(x) * ((np.pi * iteration) ** 2) / 0.0006)
            
        # if point is sampled we know the mu and have no epistemic uncertainity    
        mus, stds = _update_sampled(mus, stds, sampled, y_input)
        
        # get the uncertainity rectangles, sqrt only once here for efficiency
        lows, ups = _get_uncertainity_regions(mus, stds, np.sqrt(beta), epsilon)
        
        if iteration == 1: 
            # initialization
            rectangle_lows, rectangle_ups = lows, ups
        else:
            rectangle_lows, rectangle_ups = _union(rectangle_lows, rectangle_ups, lows, ups)
            
            
        # pareto classification, to be factored out
        # update lists 
        pareto_optimal_t, not_pareto_optimal_t, unclassified_t = _pareto_classify()
        
        
        
        # sampling from x_input 
        
        
    return pareto_optimal_t